## Data Retrieval Process

This section covers the process of retrieving the current Dow Jones Industrial Average constituents from Wikipedia and historical price data for each constituent using the yfinance library.

## Retrieving DJIA Constituents and Price Data

First, we fetch the current Dow Jones Industrial Average constituents from Wikipedia and save them to `dow_jones_constituents.csv`. Then, we fetch 10 years of daily adjusted close prices for each constituent using yfinance and save the data to `dow_jones_data.csv`. During this process, we handle any errors by printing the stock symbol that caused the error.


In [1]:

import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Fetch the Dow Jones Industrial Average constituents from Wikipedia
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
tables = pd.read_html(url)
dow_jones_constituents = tables[1] #The second table on the page contains the required data
dow_jones_constituents.to_csv('dow_jones_constituents.csv', index=False)

# Initialize an empty dataframe for storing stock data
stock_data = pd.DataFrame()

# Set the starting date for historical data (10 years ago from today)
start_date = datetime.now() - timedelta(days=365*10)

for symbol in dow_jones_constituents['Symbol']:
    try:
        ticker_data = yf.download(symbol, start=start_date)
        ticker_data['Symbol'] = symbol
        stock_data = pd.concat([stock_data, ticker_data])
    except Exception as e:
        print(f"Error occurred for symbol: {symbol}, {str(e)}")

# Save the fetched data to CSV
stock_data.to_csv('dow_jones_data.csv')

# Load the data from CSV to pandas DataFrame
dj_constituents = pd.read_csv('dow_jones_constituents.csv')
dj_data = pd.read_csv('dow_jones_data.csv')

# Replace missing data using forward fill method
dj_constituents.fillna(method='ffill', inplace=True)
dj_data.fillna(method='ffill', inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

## Implementing the Mean Reversion Strategy

Now that we have the historical price data for the Dow Jones constituents, we can proceed with implementing the mean reversion strategy. The steps involved are:

### Step 1: Calculate Daily Returns

To begin implementing the mean reversion strategy, we first need to calculate the daily returns for each stock in our `data_filled` DataFrame.


In [2]:
# Sort the DataFrame by 'Symbol' and 'Date'
dj_data.sort_values(['Symbol', 'Date'], inplace=True)

# Calculate daily returns
dj_data['Daily Return'] = dj_data.groupby('Symbol')['Adj Close'].pct_change()

# Preview the DataFrame after calculation
dj_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Daily Return
10056,2014-08-18,24.622499,24.842501,24.495001,24.790001,22.011221,190288000,AAPL,NaN
10057,2014-08-19,24.852501,25.170000,24.830000,25.132500,22.315327,277596000,AAPL,0.013816
10058,2014-08-20,25.110001,25.272499,24.987499,25.142500,22.324205,210796000,AAPL,0.000398
10059,2014-08-21,25.142500,25.235001,25.027500,25.145000,22.326427,133912000,AAPL,0.000100
10060,2014-08-22,25.072500,25.367500,25.047501,25.330000,22.490690,176736000,AAPL,0.007357


### Step 2: Identify Biggest Losers

Next, we will identify the 10 stocks with the lowest returns (biggest losers) for each trading day.


In [3]:
# Skip the first date in the dataframe as it contains only 'nan' values
dj_data_without_first_day = dj_data[dj_data['Date'] != dj_data['Date'].min()]

# Group by 'Date' and apply a lambda function to sort and get 10 first items
lowest_daily_returns = dj_data_without_first_day.groupby('Date', as_index=False).apply(lambda x: x.nsmallest(10, 'Daily Return')).reset_index(drop=True)

# View the results
lowest_daily_returns.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Daily Return
0,2014-08-19,174.910004,175.100006,173.630005,173.960007,143.432480,1916100,GS,-0.003380
1,2014-08-19,41.349998,41.480000,41.160000,41.259998,30.029032,10260400,KO,-0.002177
2,2014-08-19,34.500000,34.500000,34.250000,34.340000,26.175026,29281900,INTC,-0.002034
3,2014-08-19,49.029999,49.040001,48.270000,48.700001,29.583765,17348700,VZ,-0.001640
4,2014-08-19,133.360001,133.690002,131.800003,133.020004,101.137566,1878900,AMGN,-0.001501
5,2014-08-19,90.636841,90.713112,90.255493,90.303162,73.806137,2510884,HON,-0.000950
6,2014-08-19,92.870003,93.059998,92.720001,92.910004,73.935776,973000,TRV,-0.000860
7,2014-08-19,39.355000,39.544998,39.174999,39.255001,35.119003,4424000,NKE,-0.000509
8,2014-08-19,24.629999,24.690001,24.600000,24.639999,18.016567,20347000,CSCO,0.000406
9,2014-08-19,120.794312,121.245819,120.242477,120.610367,86.045876,1904630,MMM,0.000902


### Step 3: Simulate Trades

Now, we will simulate buying an equal amount of each of the 10 biggest losers at the close of each trading day and selling all positions at the close of the following trading day. Assume an initial capital of $100,000.


In [4]:
import pandas as pd
import numpy as np

# Ensure dates are in datetime format
lowest_daily_returns['Date'] = pd.to_datetime(lowest_daily_returns['Date'])
dj_data['Date'] = pd.to_datetime(dj_data['Date'])

# Initialize capital and results DataFrame
initial_capital = 100000
results = pd.DataFrame(columns=['Date', 'Capital'])

# Sort data by date
lowest_daily_returns = lowest_daily_returns.sort_values('Date')
dj_data = dj_data.sort_values('Date')

# Get unique dates
unique_dates = lowest_daily_returns['Date'].unique()

# Pre-calculate daily stock prices to avoid repeated lookups
dj_data_dict = {date: dj_data[dj_data['Date'] == date].set_index('Symbol')['Adj Close'] for date in unique_dates}

for i, current_date in enumerate(unique_dates[:-1]):  # Exclude last date
    next_date = unique_dates[i + 1]
    
    # Select 10 worst performers for the current date
    current_day_losers = lowest_daily_returns[lowest_daily_returns['Date'] == current_date].nsmallest(10, 'Daily Return')
    
    if len(current_day_losers) < 10:
        print(f"Skipping date {current_date} due to insufficient data")
        continue  # Skip if we don't have 10 stocks
    
    # Calculate amount to invest per stock
    amount_per_stock = initial_capital / 10
    
    total_value = 0
    for _, stock in current_day_losers.iterrows():
        symbol = stock['Symbol']
        
        # Find buy and sell prices using pre-calculated dictionary
        buy_price = stock['Adj Close']
        sell_price = dj_data_dict.get(next_date, {}).get(symbol, np.nan)
        
        if np.isnan(buy_price) or np.isnan(sell_price):
            print(f"Skipping symbol {symbol} on {current_date} due to missing price data")
            continue  # Skip if we're missing price data
        
        # Calculate number of shares and value
        shares = amount_per_stock / buy_price
        value = shares * sell_price
        
        total_value += value
    
    # Update capital
    initial_capital = total_value if total_value > 0 else initial_capital
    
    # Store results
    new_row = pd.DataFrame({'Date': [next_date], 'Capital': [initial_capital]})
    results = pd.concat([results, new_row], ignore_index=True)

    print(f"Processed {current_date}, Capital: {initial_capital:.2f}")

# Calculate and print results
final_capital = results['Capital'].iloc[-1]
total_return = (final_capital - 100000) / 100000 * 100

print(f"Final Capital: ${final_capital:.2f}")
print(f"Total Return: {total_return:.2f}%")

C:\Users\ashia\AppData\Local\Temp\ipykernel_26832\2658692867.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)


Processed 2014-08-19 00:00:00, Capital: 100343.05
Processed 2014-08-20 00:00:00, Capital: 100728.19
Processed 2014-08-21 00:00:00, Capital: 100783.45
Processed 2014-08-22 00:00:00, Capital: 101215.64
Processed 2014-08-25 00:00:00, Capital: 101230.73
Processed 2014-08-26 00:00:00, Capital: 101564.18
Processed 2014-08-27 00:00:00, Capital: 101095.42
Processed 2014-08-28 00:00:00, Capital: 101244.02
Processed 2014-08-29 00:00:00, Capital: 101503.83
Processed 2014-09-02 00:00:00, Capital: 101583.70
Processed 2014-09-03 00:00:00, Capital: 102176.13
Processed 2014-09-04 00:00:00, Capital: 102645.32
Processed 2014-09-05 00:00:00, Capital: 102802.00
Processed 2014-09-08 00:00:00, Capital: 101652.42
Processed 2014-09-09 00:00:00, Capital: 102490.13
Processed 2014-09-10 00:00:00, Capital: 102460.79
Processed 2014-09-11 00:00:00, Capital: 102042.79
Processed 2014-09-12 00:00:00, Capital: 101757.13
Processed 2014-09-15 00:00:00, Capital: 102458.11
Processed 2014-09-16 00:00:00, Capital: 102646.08


### Step 4: Calculate Performance Metrics

Finally, we will calculate the strategy's annualized return, annualized volatility, Sharpe ratio (assume a risk-free rate of 0), and maximum drawdown.


In [5]:
# Calculation of Performance Metrics

# Assuming 252 trading days in a year
trading_days = 252

# Compute daily capital returns
results['Capital Return'] = results['Capital'].pct_change() 

# Annualized return
annual_return = (1 + results['Capital Return'].mean())**trading_days - 1

# Annualized volatility
annual_volatility = results['Capital Return'].std() * (trading_days**0.5)

# Sharpe Ratio assuming risk-free rate = 0
sharpe_ratio = annual_return / annual_volatility

# Printing the calculated metrics
print(f"Annualized Return : {annual_return*100:.2f}% ")
print(f"Annualized Volatility : {annual_volatility*100:.2f}% ")
print(f"Sharpe Ratio : {sharpe_ratio:.2f} ")

Annualized Return : 15.72% 
Annualized Volatility : 20.08% 
Sharpe Ratio : 0.78 


### Step 5: Compare with Dow Jones Index

To determine if our mean reversion strategy outperformed the market, we will compare its Sharpe ratio with that of the Dow Jones Index. We'll use the SPDR Dow Jones Industrial Average ETF Trust (DIA) as a proxy for the Dow Jones. The point here is that we want to find out if betting on the losers of the Dow Jones, rather than the Dow Jones itself, is a more profitabl estrategy in hindsight. 


In [6]:
import numpy as np

# Fetch the DIA ETF data from Yahoo Finance
dia_data = yf.download('DIA', start=start_date)

# Calculate the daily returns for DIA
dia_data['Daily Return'] = dia_data['Adj Close'].pct_change()

# Calculate the Sharpe ratio for DIA
annual_return_dia = (1 + dia_data['Daily Return'].mean())**trading_days - 1
annual_volatility_dia = dia_data['Daily Return'].std() * np.sqrt(trading_days)
sharpe_ratio_dia = annual_return_dia / annual_volatility_dia

print(f"Sharpe Ratio for DIA: {sharpe_ratio_dia:.2f} ")

# Compare Sharpe ratios
if sharpe_ratio > sharpe_ratio_dia:
    print("Our mean reversion strategy outperformed the general Dow Jones.")
else:
    print("Our mean reversion strategy did not outperform the general Dow Jones.")

[*********************100%%**********************]  1 of 1 completed

Sharpe Ratio for DIA: 0.75 
Our mean reversion strategy outperformed the general Dow Jones.


### Step 6: Compare Our Mean Reversion Strategies' Performance to that of the Dow Jones ETF

To better understand the performance of our mean reversion strategy compared to investing in the Dow Jones, we will visualize the annual returns, standard deviations, and Sharpe ratios of both strategies.


In [7]:
# Gathering calculated data for mean reversion strategy and DIA
data = {
    'Mean Reversion Strategy': [annual_return, annual_volatility, sharpe_ratio],
    'DIA': [annual_return_dia, annual_volatility_dia, sharpe_ratio_dia]
}

# Creating a pandas DataFrame with this data
df = pd.DataFrame(data, index=['Annual Return', 'Annual Std Dev', 'Sharpe Ratio'])

# Printing the DataFrame
print(df)

# Finding and printing the strategy with the higher Sharpe Ratio
higher_sharpe_ratio_strategy = df.idxmax(axis=1)['Sharpe Ratio']
print(f"\nStrategy with higher Sharpe Ratio: {higher_sharpe_ratio_strategy}")

                Mean Reversion Strategy       DIA
Annual Return                  0.157172  0.130327
Annual Std Dev                 0.200779  0.174573
Sharpe Ratio                   0.782809  0.746550

Strategy with higher Sharpe Ratio: Mean Reversion Strategy


### Step 7: Visualize Portfolio Growth

To better understand the performance of our mean reversion strategy compared to investing in the Dow Jones, we will visualize the growth of a hypothetical 100,000 dollar portfolio over time for both strategies.


In [8]:
import plotly.graph_objects as go

# Calculate Cumulative Returns by adding 1 to daily returns, then calculate the cumulative product
results['Cumulative Return'] = (1 + results['Capital Return']).cumprod()
dia_data['Cumulative Return'] = (1 + dia_data['Daily Return']).cumprod()

# Calculate Portfolio Value by multiplying Cumulative Returns by initial capital
results['Portfolio Value'] = results['Cumulative Return'] * 100000
dia_data['Portfolio Value'] = dia_data['Cumulative Return'] * 100000

# Create Line plots 
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results['Date'], 
    y=results['Portfolio Value'], 
    mode='lines', 
    name='Mean Reversion Strategy'
))

fig.add_trace(go.Scatter(
    x=dia_data.index, 
    y=dia_data['Portfolio Value'], 
    mode='lines', 
    name='DIA ETF'
))

# Add titles and labels
fig.update_layout(
    title='Growth of $100,000 Portfolio Over Time',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    legend_title='Strategy',
    autosize=False,
    width=1000, 
    height=500
)

# Display Plot
fig.show()